### Model Selection 학습

학습/테스트 데이터 : train_test_split()

model select 모듈 - 학습 데이터와 텍스트 데이터 세트를 분리 또는 교차 검증분할 및 평가, 하이퍼 파라미터 튜닝을 위한 다양한 함수 제공

In [25]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


**model selection 로직 비적용한 논리적 모델링 오류**

In [26]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()

train_data = iris.data  # 독립변수, feature, 
train_label = iris.target # 종속변수, label, class, target

# 학습
dt_clf.fit(train_data, train_label)

# 예측 해 주세요
pred = dt_clf.predict(train_data)

# 예측된 결과가 100% 일치되었다란 의미 
print('예측 정확도: ', accuracy_score(train_label, pred) )

예측 정확도:  1.0


<hr>

**1차 개선 : train_test_split()로 학습, 테스트 데이터 분할**

In [27]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()

iris_data = iris.data  # 독립변수, feature, 
iris_label = iris.target # 종속변수, label, class, target

X_train, X_test, y_train, y_test = train_test_split(iris_data, iris_label, test_size=0.4, random_state=121)
dt_clf = DecisionTreeClassifier(random_state=11)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도 : {0:.4f}'.format(accuracy_score(y_test, pred)))

예측 정확도 : 0.9667


In [28]:
train_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

<hr>

**2차 개선 : 교차 검증**

In [29]:
from sklearn.model_selection import KFold
import numpy as np

In [30]:
iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier()

# 5개의 폴드 셋트로 분할
kfold = KFold(n_splits=5)

# 평가한 결과치를 적재하게 되는 리스트
cv_accuracy = []

In [31]:
n_iter = 0

# 폴드별 학습용, 검증용 row index를 array 반환
for train_index, test_index in kfold.split(features):
    
    # 학습용, 검증용 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]

    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter += 1
    
    # 반복시마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    cv_accuracy.append(accuracy)
    
print(np.mean(cv_accuracy))    
cv_accuracy = []

0.91334


<hr>

**Stratified K 폴드**

- 불균형한(imbalanced) 분포도를 가진 레이블(결정 class) 데이터 집합을 위한 K 폴드 방식
- 불균형한 분포도란? 레이블 데이터 집합은 특정 레이블 값이 특이하게 많거나 매우 적어서 값의 분포가 한쪽으로 치우치는 것 의미
- K 폴드가 레이블 데이터 집합이 원본 데이터 집합의 레이블 분포를 학습 및 테스트 세트에 제대로 분배하지 못하는 경우의 문제 해결
- 원리 : 원본 데이터의 레이블(target) 분포를 먼저 고려 -> 이 분포와 동일하게 학습과 검증 데이터 세트를 분배

<hr>

**용어**

1. feature = 특성 = 속성 = 독립변수 <br>
2. label = class = target = 종속변수

In [32]:
import pandas as pd

In [33]:
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target


In [34]:
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [35]:
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


**학습/검증 레이블 데이터 값의 분포도 확인**

분포도 불균형 확인 <br>
학습 데이터로 사용되지 않은 이상 예측이 부정확 <br>
해결책 : StratifiedKFold API 활용 

In [36]:
kfold = KFold(n_splits=3)

n_tier = 0
for train_index, test_index in kfold.split(iris_df):
    n_tier += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
#     print('* 교차 검증 : {0}'.format(n_tier))
    print("--"*20)
    print('학습 레이블 분포 : \n', label_train.value_counts())
    print('검증 레이블 분포 : \n', label_test.value_counts())
    

----------------------------------------
학습 레이블 분포 : 
 2    50
1    50
Name: label, dtype: int64
검증 레이블 분포 : 
 0    50
Name: label, dtype: int64
----------------------------------------
학습 레이블 분포 : 
 2    50
0    50
Name: label, dtype: int64
검증 레이블 분포 : 
 1    50
Name: label, dtype: int64
----------------------------------------
학습 레이블 분포 : 
 1    50
0    50
Name: label, dtype: int64
검증 레이블 분포 : 
 2    50
Name: label, dtype: int64


<hr>

**학습/검증 레이블 데이터 값의 분포도 확인후 불균형 분포 해결**

split() 시에는 target(종속변수,label,class)의 정보도 반드시 적용!!!

In [37]:
from sklearn.model_selection import StratifiedKFold

In [38]:
skf = StratifiedKFold(n_splits=3)

n_tier = 0
for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_tier += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    
    print("--"*20)
    print('학습 레이블 분포 : \n', label_train.value_counts())
    print('검증 레이블 분포 : \n', label_test.value_counts())
    

----------------------------------------
학습 레이블 분포 : 
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 분포 : 
 1    17
0    17
2    16
Name: label, dtype: int64
----------------------------------------
학습 레이블 분포 : 
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 분포 : 
 2    17
0    17
1    16
Name: label, dtype: int64
----------------------------------------
학습 레이블 분포 : 
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 분포 : 
 2    17
1    17
0    16
Name: label, dtype: int64


In [39]:
# ? StratifiedKFold 활용 모델만들기

In [71]:
# 붓꽃 데이터 세트에서 Stratified Kfold 를 이용한 검증

dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy = []

# StratifiedKFold의 split 호출 시 반드시 레이블 데이터 세트도 추가 입력 필요
for train_index, test_index in skfold.split(features, label):
    #split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    # 반복 시마다 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('{0} 교차검증 정확도: {1}, 학습데이터 크기: {2}, 검증데이터 크기: {3}'.format(n_iter, accuracy, train_size, test_size))
    cv_accuracy.append(accuracy)
    
# 교차검증별 정확도 및 평균정확도 계산
print('\n 교차검증별 정확도: ', cv_accuracy)
print('## 평균 검증 정확도: ', np.mean(cv_accuracy))

1 교차검증 정확도: 0.98, 학습데이터 크기: 100, 검증데이터 크기: 50
2 교차검증 정확도: 0.94, 학습데이터 크기: 100, 검증데이터 크기: 50
3 교차검증 정확도: 0.98, 학습데이터 크기: 100, 검증데이터 크기: 50

 교차검증별 정확도:  [0.98, 0.94, 0.98]
## 평균 검증 정확도:  0.9666666666666667


<hr>

## **교차 검증을 더 간결하게 작업하기 : cross_val_score()**

1. 교차 검증을 쉽게 해주는 API
2. 폴드셋 설정 -> for에서 반복 학습 및 테스트 데이터의 index 추출 -> 반복적으로 학습과 예측 수행 및 성능 평가 : 한번에 처리

In [69]:
from sklearn.model_selection import cross_val_score, cross_validate
dt_clf = DecisionTreeClassifier(random_state=100)

data = iris.data
label = iris.target

scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=5)
print('교차 검증 정확도:', np.round(scores,4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증 정확도: [0.9667 0.9667 0.9    0.9333 1.    ]
평균 검증 정확도: 0.9533


In [70]:
from sklearn.model_selection import cross_val_score, cross_validate
dt_clf = DecisionTreeClassifier(random_state=88)

data = iris.data
label = iris.target

#cv로 지정된 횟수만큼 scoring 파라미터로 지정된 평가 결과값 도출
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print('교차 검증 정확도:', np.round(scores,4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증 정확도: [0.98 0.92 1.  ]
평균 검증 정확도: 0.9667


**교차 검증과 최적 하이퍼 파라미터 튜닝을 한번에 하기**

1. GirdSearchCV
2. 하이퍼 파라미터를 순차적으로 적용하면서 편리하게 최적의 파라미터 도출
3. 격자형식으로 dict 구조의 하이퍼파라미터값 적용

In [73]:
'''
1과 2
1과 3

2와 2
2와 3

3과 2
3돠 3
'''
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

In [77]:
from sklearn.model_selection import GridSearchCV

iris = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size = 0.2, random_state=121)

dtree = DecisionTreeClassifier()

# 딕셔너리 형식의 파라미터
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

#refit=True가 default, True인 경우 가장 좋은 parameter 설정으로 재 학습
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

#학습을 통한 모델 개발
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과를 추출해서 DataFrame 변환
score_df = pd.DataFrame(grid_dtree.cv_results_)
score_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]


,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [78]:
print('GridSearch 최적 파라미터: ', grid_dtree.best_params_)
print('GridSearch 최고 점수: ', grid_dtree.best_score_)

GridSearch 최적 파라미터:  {'max_depth': 3, 'min_samples_split': 2}
GridSearch 최고 점수:  0.975
